# Quick example of implementing a simple payments network

There are 4 steps to running Veriphi guardrails on a tool given to an autonomous Agent.
 - Define network classes
 - Prepare data
 - Distribute and encrypt
 - Reconstruct and recover 

In [1]:
import os
import json
import numpy as np

from veriphi_core import utils as utils
from veriphi_core import interface as ic

In [2]:
net_dict = {"Agent": 0, 
            "Authoriser": 1, 
            "Endpoint": 2}

## Prepare data

In [3]:
wallet = {
    "walletId": "abc123",
    "owner": {
        "name": "Jane Doe",
        "id": "user-678",
    },
    "defaultCurrency": "USD",
    "addresses": {
        "BTC": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa",
        "ETH": "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7",
    },
}

# Serialize to JSON string
wallet_json = json.dumps(wallet)

# Convert to UTF-8 bytes (like TextEncoder in TS)
wallet_bytes = wallet_json.encode("utf-8")

In [4]:
wallet_bytes

b'{"walletId": "abc123", "owner": {"name": "Jane Doe", "id": "user-678"}, "defaultCurrency": "USD", "addresses": {"BTC": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa", "ETH": "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7"}}'

In [5]:
len(wallet_bytes)

210

In [6]:
# Generate a public key for distribution and a private key for obfuscation
setupNode = ic.SetupNode('node_setup')
seed = np.frombuffer(os.urandom(32),dtype=np.uint8)
pub_key = setupNode.gen_public_key(seed)
priv_key = setupNode.gen_private_key('obf_private_key',seed)
encrypted, nonce = setupNode.encrypt_data(wallet_bytes, priv_key)
low_val, high_val = setupNode.implement_conditions(0.0,1000.0,priv_key)
encrypted, chunk_size = setupNode.obfuscate_data(np.frombuffer(encrypted,dtype=np.uint8), priv_key, low_val, high_val, 50.0)

### The above cell can be wrapped up in a single function call

In [7]:
public_data, private_data = ic.setup_node(wallet_bytes,0,1000,True)

In [8]:
# pack the data for transmission
agent_packet    = setupNode.package_data(encrypted, pub_key,'E',net_dict["Agent"])
auth_packet     = setupNode.package_data(encrypted, pub_key,'E',net_dict["Authoriser"])
endpoint_packet = setupNode.package_data(encrypted, pub_key,'E',net_dict["Endpoint"])

### More generally for many parties

In [9]:
agent_packet, auth_packet, endpoint_packet = ic.distribute_data(public_data,'E',3)

## Distribute and encrypt

### Example code for Agent members

In [10]:
agent_node = ic.EncryptNode('agent_member')
public_key, packet_data, mode, identity = agent_node.unpackage_data(agent_packet)
agent_priv_key = agent_node.gen_private_key('private_key', np.frombuffer(os.urandom(32),dtype=np.uint8))
agent_encrypted = agent_node.encrypt_data(packet_data, agent_priv_key, public_key, mode, identity)
agent_enc_packet = agent_node.package_data(agent_encrypted, mode, identity)

# Or with simple function calls

In [11]:
agent_enc_packet = ic.encrypt_node(agent_packet,"agent_member")
auth_enc_packet  = ic.encrypt_node(auth_packet,"authoriser_member")
endp_enc_packet  = ic.encrypt_node(endpoint_packet,"endpoint_member")

## Reconstruction and recovery is conditioned on complete visibility and correct conditions

In [12]:
veriphier = ic.DecryptNode('Veriphier')
party_data = veriphier.collect_packets(agent_enc_packet, auth_enc_packet, endp_enc_packet)
party_data_recov = veriphier.recover_packets(party_data)
stream_list = veriphier.reconstruct_data(party_data_recov)
reconstructed = veriphier.reassemble_data(stream_list, party_data[0]['mode'])
recovered, chunk_size = veriphier.obfuscate_data(reconstructed, private_data["key"], private_data["low_val"], private_data["high_val"], 500)
decrypted = veriphier.decrypt_data(recovered.tobytes(), private_data["nonce"], private_data["key"])

In [13]:
chunk_size

5

In [14]:
recovered

array([ 87, 135, 136, 239,  45, 203, 177,  87, 158,  34, 136,  20, 123,
       191, 106,  15, 231, 116, 196, 139, 132, 228, 205,  94, 167, 150,
        99, 172,  50, 175, 106, 150, 236, 206, 144, 236, 191, 208, 138,
       231, 158, 192, 237,  27, 186, 191, 196,  27, 232, 183,  24, 170,
       224, 191,   4,   7, 196, 233,  82, 116, 121,  69,  94, 146,  31,
       191,  13, 133, 216, 104, 118, 246, 229, 136,  48, 240,  37, 216,
       167,  63, 136, 241, 250, 237, 109,  11, 137, 212, 170,   2, 227,
       117,  59, 126,  81,  56, 205, 140, 234, 152, 122, 141, 218, 134,
        48, 239, 116,  59,  63,  31, 213,  91,  80, 247, 157, 216, 162,
       119, 108, 218,  97, 130,  85, 148, 191, 252,  82,  49, 244, 243,
        93, 122,  39,  19,  74,  59, 228, 255,  63, 222, 160, 107, 181,
        52,  81,  96,  32, 203,  75,  34, 132, 129, 110, 104, 148, 148,
         1, 206,  34,  52, 213, 255, 186, 214,  20, 249, 251,  44,  32,
       221, 108,  24, 223,   9, 191,  21, 180, 127, 152, 111, 11

### Or wrapped up in a single call

In [15]:
decrypted = ic.decrypt_node(private_data, 500, True, agent_enc_packet, auth_enc_packet, endp_enc_packet)

In [16]:
recovered_wallet = decrypted.decode("utf-8")

In [17]:
recovered_wallet

'{"walletId": "abc123", "owner": {"name": "Jane Doe", "id": "user-678"}, "defaultCurrency": "USD", "addresses": {"BTC": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa", "ETH": "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7"}}'